**Querying_netcdf_with_PYTHON_KDTREE**
- To query a netcdf with latitude and longitude is required for objective three, in which user pointed lat long, revived as SMS from Android app has to parsed and find its model and nearest dylos monitoring station to send replay.
- there is very useful [tutorial](http://nbviewer.ipython.org/github/Unidata/tds-python-workshop/blob/master/netcdf-by-coordinates.ipynb) on this with elaboration on different implementations advantages 
- In which most advanced querying was based on KDtree, this implementation was used to query netcdf generated from WRF model the code is as follows
```python
import numpy as np
import netCDF4
from math import pi
from numpy import cos, sin
from scipy.spatial import cKDTree
class Kdtree_fast(object):
    def __init__(self, ncfile, latvarname, lonvarname):
        self.ncfile = ncfile
        self.latvar = self.ncfile.variables[latvarname]
        self.lonvar = self.ncfile.variables[lonvarname]        
        # Read latitude and longitude from file into numpy arrays
        rad_factor = pi/180.0 # for trignometry, need angles in radians
        self.latvals = self.latvar[:] * rad_factor
        self.lonvals = self.lonvar[:] * rad_factor
        self.shape = self.latvals.shape
        clat,clon = cos(self.latvals),cos(self.lonvals)
        slat,slon = sin(self.latvals),sin(self.lonvals)
        clat_clon = clat*clon
        clat_slon = clat*slon
        triples = zip(np.ravel(clat*clon), np.ravel(clat*slon), np.ravel(slat))
        self.kdt = cKDTree(triples)
    def query(self,lat0,lon0):
        rad_factor = pi/180.0 
        lat0_rad = lat0 * rad_factor
        lon0_rad = lon0 * rad_factor
        clat0,clon0 = cos(lat0_rad),cos(lon0_rad)
        slat0,slon0 = sin(lat0_rad),sin(lon0_rad)
        dist_sq_min, minindex_1d = self.kdt.query([clat0*clon0,clat0*slon0,slat0])
        iy_min, ix_min = np.unravel_index(minindex_1d, self.shape)
        return iy_min,ix_min
ncfile = netCDF4.Dataset('/home/swl-sacon-dst/Documents/GISE_2013/LAB/GRIBJSON_inst/wrfncxnj-0.1_r2120/wrfout_d04_2014-03-01_01:00:00_tas.nc', 'r')
ns = Kdtree_fast(ncfile,'lat','lon')
iy,ix = ns.query(8.47, 76.95)
tas = ncfile.variables['tas']
print 'Closest lat lon:', ns.latvar[iy,ix], ns.lonvar[iy,ix]
print 'Temperature:', tas[0, 0, iy, ix], tas.units
ncfile.close()
```